<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/SpellingQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spelling Quiz

In [ ]:
!pip install gradio pandas gtts

## Upload a csv file with at least two columns: Levels, Words

Levels should have Level1 or Level2
[sample file](https://raw.githubusercontent.com/MK316/Myapps/main/data/wordlist.csv)

In [ ]:
import pandas as pd

# Make sure to replace the URL below with the URL of your raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/MK316/Myapps/main/data/wordlist.csv'
df = pd.read_csv(url)


## Or you can upload your file (from your computer)

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=filename, length=len(uploaded[filename])))


## Below is an app using Gradio

In [ ]:
#@markdown SpellingQuiz
import gradio as gr
import pandas as pd
import random
from gtts import gTTS
import os

# Load the DataFrame globally
df = pd.read_csv('/content/wordlist.csv')

# Function to generate audio from a word
def generate_audio(word):
    tts = gTTS(word, lang='en')
    audio_file = '/tmp/word.mp3'
    tts.save(audio_file)
    return audio_file

# Function to get a random word from a level that hasn't been used yet
def get_random_word(level, used_words):
    level_df = df[(df['Levels'] == level) & (~df['Words'].isin(used_words))]
    if level_df.empty:
        return None, "Game over!"
    word = level_df.sample().iloc[0]['Words']
    return word, ""

# Function to start the game
def start_game(level):
    word, message = get_random_word(level, [])
    if message:
        return None, message, "", {'score': 0, 'total': 0, 'level': level, 'used_words': []}
    audio_file = generate_audio(word)
    game_state = {'score': 0, 'total': len(df[df['Levels'] == level]), 'level': level, 'used_words': [word]}
    return audio_file, "", word, game_state

# Function to process the user's spelling input
def process_spelling(user_input, game_state):
    current_word = game_state['used_words'][-1]
    if user_input.strip().lower() == current_word.lower():
        game_state['score'] += 1
        result = "Correct!"
    else:
        result = f"Incorrect. The correct spelling was: {current_word}"

    # Get the next word that hasn't been used yet
    word, message = get_random_word(game_state['level'], game_state['used_words'])
    if message:
        final_result = f"Game over! Your score is {game_state['score']} out of {game_state['total']}"
        return None, final_result, "", {}

    # Generate audio for the next word and add it to the used words list
    audio_file = generate_audio(word)
    game_state['used_words'].append(word)
    return audio_file, result, word, game_state

# Define the Gradio Blocks
with gr.Blocks() as demo:
    with gr.Row():
        level = gr.Radio(choices=['Level1', 'Level2'], label="Select Level")
        start_btn = gr.Button("Start")
    with gr.Row():
        audio = gr.Audio(label="Listen to the Word")
        user_input = gr.Textbox(label="Type the spelling here")
        submit_btn = gr.Button("Submit")
    result = gr.Textbox()
    current_word = gr.Textbox(visible=False)
    game_state = gr.State()

    start_btn.click(start_game, inputs=level, outputs=[audio, result, current_word, game_state])
    submit_btn.click(process_spelling, inputs=[user_input, game_state], outputs=[audio, result, current_word, game_state])

demo.launch()
